# Smoke Alarm install model
### this notebook represents the current smoke alarm install model

In [66]:
import pandas as pd
import os
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt 
import missingno as msno
import geopandas as gpd

In [68]:
# import stored methods 
path = Path.cwd().parent.parent
LoadAndCleanACS = path /'src' /'data'/ 'LoadAndCleanACS.py'
LoadAndCleanARCP = path /'src' /'data'/ 'LoadAndCleanARCP.py'


In [69]:
# Run methods to obtain clean datasets 
%run $LoadAndCleanACS
%run $LoadAndCleanARCP



In [70]:
base_path = Path.cwd().parent.parent
shp_path =  base_path / 'Data' / 'Master Project Data' / 'Tiger_censusBlocks_2016'
shp_files = shp_path.glob('**/*.zip')
shapes = gpd.GeoDataFrame()
for z in shp_files:
    shapes = shapes.append( gpd.read_file( 'zip://' + str(z) ) )  


In [71]:
# to allow for all variables to be displayed in jupyter
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [72]:
def StandardizeColumnNames(df):
    """
    Standardizes column names
    """
    df.columns = map(str.lower, df.columns)
    df.columns = df.columns.str.replace(', ', '_')
    df.columns = df.columns.str.replace('-', '_')
    df.columns = df.columns.str.replace('/', '_')
    df.columns = df.columns.str.replace('(', '_')
    df.columns = df.columns.str.replace(')', '_')
    df.columns = df.columns.str.replace(' ', '_')
    #print(df.columns)
    return df

### Data

In [73]:
input_loc =  path /'Data'/ 'Master Project Data'
output_loc = path /'Data'/ 'processed'

In [74]:
arc_path = input_loc / 'ARC Preparedness Data.csv'
arc = pd.read_csv(arc_path, 
                  dtype = {'GEOID': str, 'Zip': str})
arc = StandardizeColumnNames(arc)
arc.dropna(inplace = True)
# trim geoid leading saftey marks 
arc['geoid'] = arc['geoid'].str[2:]


## EDA  

- remove all houses that don't have a previous smoke detector record 
- Determine the median number of house visist
- Visualize visit distribution 
- use ACS data to determine % of blocks visited 
- determine % blocks visited with >30 visits 

In [75]:
arc

,geoid,census_block_group_y,census_block_group_x,city,state,zip,county,in_home_visit_date,smoke_alarms_installed__9_volt_10_year_dhh_,10_year_and_9_volt_alarms_installed,dhh_alarms_installed,pre_existing_alarms,pre_existing_alarms_tested_and_working,batteries_replaced,fire_escape_plans_made,fire_safety_checklists_completed,additional_hazard_education_conducted,additional_hazard_type,people_served,youth_served,seniors_served,veterans_military_members_and_military_family_members_served,individuals_with_disabilities_access_or_functional_needs_served
12,010010205002,32.470418,-86.424166,PRATTVILLE,AL,36066,Autauga,9/9/2016,1,1,0,0.0,0.0,0,1,1,0,None,2,0,0,0,0
18,010010208012,32.455173,-86.534591,PRATTVILLE,AL,36067,Autauga,9/30/2019,2,2,0,0.0,0.0,0,1,1,1,Tornadoes,5,0,0,1,0
23,010010208021,32.524822,-86.573009,PRATTVILLE,AL,36067,Autauga,5/4/2019,3,2,1,0.0,0.0,0,1,1,1,Other,1,0,0,0,0
24,010010208021,32.524822,-86.573009,PRATTVILLE,AL,36067,Autauga,9/27/2019,1,1,0,0.0,0.0,0,1,1,1,Tornadoes,1,0,0,0,0
27,010010208023,32.543983,-86.492100,PRATTVILLE,AL,36067,Autauga,12/2/2017,3,3,0,0.0,0.0,0,1,1,0,None,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862041,560459513003,43.843551,-104.260072,NEWCASTLE,WY,82701,Weston,3/4/2017,2,2,0,5.0,3.0,0,1,1,1,Wildfires,1,0,0,0,0
862042,560459513003,43.843551,-104.260072,NEWCASTLE,WY,82701,Weston,3/4/2017,3,3,0,0.0,0.0,0,1,1,1,Wildfires,4,2,0,0,0
862043,560459513003,43.843551,-104.260072,NEWCASTLE,WY,82701,Weston,3/4/2017,3,3,0,5.0,0.0,0,1,1,1,Wildfires,2,0,1,0,1
862044,560459513003,43.843551,-104.260072,NEWCASTLE,WY,82701,Weston,3/4/2017,4,4,0,1.0,0.0,0,1,1,1,Wildfires,3,0,0,0,0


In [76]:
#block level
counts = arc['geoid'].value_counts()
counts_median = counts.median()
counts.describe()


count    87725.000000
mean         6.132619
std         15.250533
min          1.000000
25%          1.000000
50%          2.000000
75%          6.000000
max       1863.000000
Name: geoid, dtype: float64

## EDA- Geograpic Level
 repeat block Level analysis at various levels

In [77]:
# county
county_counts =  arc['geoid'].str[:7].value_counts()
print('County Level')
print(county_counts.describe())
# state 
state_counts =  arc['geoid'].str[:2].value_counts()
print('\n State Level')
print(state_counts.describe())

County Level
count    3605.000000
mean      149.232732
std       331.121218
min         1.000000
25%        10.000000
50%        47.000000
75%       148.000000
max      6579.000000
Name: geoid, dtype: float64

 State Level
count       52.000000
mean     10345.846154
std      12099.256714
min          3.000000
25%       2130.500000
50%       7135.500000
75%      12123.000000
max      51826.000000
Name: geoid, dtype: float64


In [78]:
print(arc['pre_existing_alarms'].describe())
print(arc['pre_existing_alarms_tested_and_working'].describe())

count    537984.000000
mean          1.507301
std           1.552540
min           0.000000
25%           0.000000
50%           1.000000
75%           2.000000
max           9.000000
Name: pre_existing_alarms, dtype: float64
count    537984.000000
mean          0.829638
std           1.336587
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           9.000000
Name: pre_existing_alarms_tested_and_working, dtype: float64


## Confidence Interval Motivation 

A commonly used formula for a binomial confidence interval relies on approximating the distribution of error about a binomially-distributed observation, ${\displaystyle {\hat {p}}}$, with a normal distribution. This approximation is based on the central limit theorem and is unreliable when the sample size is small or the success probability is close to 0 or 1.

Using the normal approximation, the success probability p is estimated as

${\displaystyle {\hat {p}}\pm z{\sqrt {\frac {{\hat {p}}\left(1-{\hat {p}}\right)}{n}}},}$

Source https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval

In [79]:
def CreateConfidenceIntervals(num_surveys,percentage):
# this function takes the cleaned data and adds a confidence interval 

    z =	1.960 # corresponds to 95% confidence interval
    
    CI =  z * np.sqrt(
                     (percentage * (100 - percentage) ) / 
                      num_surveys  )

    return CI


### Feature Engineering
 Create Binary variables out of the smoke_alarms_present and smoke_alarms_tested_and_working variables 
 
 We'll then create a new dataset with the aggregated number and percantage of working smoke detectors in each census geography

In [80]:
def CreateSingleLevelSmokeAlarmModel(df,geo_level, acs = ACS.copy() ):
# This function takes the arc data  into a dataset containing the percentage 
# and number of smoke detectors by census geography
#
# Inputs 
# arc-  the arc dataset
#
# geo_level- String var indcating what census geography to aggregate on. current levels are:
# State,County,Block,State
#
# The resultant dataset will have the following values:
#
#   num_surveys - total number of surveys conducted
#
#   detectors_found -   houses with at least one smoke detector in the home
#
#   detectors_workding - houses with at least one tested and working smoke detector in the home
#
#   Note: for variables the suffixes 
#       _total- indicates raw counts 
#        _prc  - indicates percentage: (_total / num_surveys * 100)
#
   
    # dict with relevant length of GEOID for tract geography
    geo_level_dict = {'State':2,'County':5,'Tract':11,'Block':12}
    
    df['geoid'] = df['geoid'].str[: geo_level_dict[geo_level]]
    acs.index =  acs.index.str[:geo_level_dict[geo_level]]
    acs.drop_duplicates(inplace = True)
    ## binarize pre_existing_alarms and _tested_and_working
    #  values will now be: 0 if no detectors present and 1 if any number were present
    df['pre_existing_alarms'].where(df['pre_existing_alarms'] < 1, other = 1, inplace = True) 
    df['pre_existing_alarms_tested_and_working'].where(
                                                        df['pre_existing_alarms_tested_and_working'] < 1,
                                                            other = 1, 
                                                            inplace = True)

    ## create detectors dataset
    # This happens by grouping data both on pre_existing alarms and then _tested_and working alarms 
    # and then merging the two into the final dataset

    detectors =  df.groupby('geoid')['pre_existing_alarms'].agg({np.size ,
                                                                  np.sum,
                                                                  lambda x: np.sum(x)/np.size(x)* 100 })

    detectors.rename({'size':'num_surveys','sum':'detectors_found_total','<lambda_0>':'detectors_found_prc'},
                     axis =1,
                     inplace = True)

    detectors['detectors_found_prc'] = detectors['detectors_found_prc'].round(2)
    
  
    
    d2 =  df.groupby('geoid')['pre_existing_alarms_tested_and_working'].agg({np.size,np.sum, 
                                                                              lambda x: np.sum(x)/np.size(x)* 100 })
    
    d2.rename({'size':'num_surveys2','sum':'detectors_working_total','<lambda_0>':'detectors_working_prc'},
                     axis =1,
                     inplace = True)

    
    d2['detectors_working_prc'] = d2['detectors_working_prc'].round(2)
    

    detectors = detectors.merge(d2,how = 'left', on ='geoid')

    detectors['detectors_found_CI'] = CreateConfidenceIntervals(detectors['num_surveys'].values,
                                                                detectors['detectors_found_prc'].values )
                                                                
    detectors['detectors_working_CI'] = CreateConfidenceIntervals(detectors['num_surveys'].values,
                                                                detectors['detectors_working_prc'].values )  
    
    
    
    
    
    # rearrange columns 
    column_order = ['num_surveys',	
                    'detectors_found_total',
                    'detectors_found_prc', 
                    'detectors_found_CI',
                    'detectors_working_total',
                    'detectors_working_prc',
                    'detectors_working_CI']
    
    detectors = detectors[column_order]
    
    detectors = detectors[~pd.isna(detectors.index)]
# fix block model to ensure blocks that weren't visited are added to the model 
    detectors = detectors.reindex(detectors.index.union(acs.index.unique()),fill_value = 0)
    detectors = detectors[~pd.isna(detectors.index)]
   

# test if there are missing values in resultant 

    return detectors

In [291]:
 arc_state = CreateSingleLevelSmokeAlarmModel(arc.copy(),'State',ACS.copy())
 arc_state.head()

,num_surveys,detectors_found_total,detectors_found_prc,detectors_found_CI,detectors_working_total,detectors_working_prc,detectors_working_CI
01,6975.0,4071.0,58.37,1.156863,2189.0,31.38,1.089020
02,2172.0,1532.0,70.53,1.917358,717.0,33.01,1.977670
04,6366.0,3363.0,52.83,1.226298,1443.0,22.67,1.028543
05,4503.0,2442.0,54.23,1.455175,1305.0,28.98,1.325088
06,44657.0,32376.0,72.50,0.414139,18790.0,42.08,0.457892


In [292]:
arc_county = CreateSingleLevelSmokeAlarmModel(arc.copy(),'County',ACS.copy())
arc_county.describe()

,num_surveys,detectors_found_total,detectors_found_prc,detectors_found_CI,detectors_working_total,detectors_working_prc,detectors_working_CI
count,3143.000000,3143.000000,3143.000000,3143.000000,3143.000000,3143.000000,3143.000000
mean,171.168947,114.517340,53.793312,10.515280,68.384664,30.944578,10.772218
std,544.821352,378.349203,31.853492,12.420254,252.672568,24.790769,12.876119
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,2.000000,33.590000,0.000000,1.000000,9.090000,0.000000
50%,34.000000,20.000000,61.620000,7.250781,10.000000,30.770000,7.332109
75%,133.500000,84.000000,77.780000,14.176551,47.500000,46.400000,14.099762
max,12981.000000,8841.000000,100.000000,69.296465,7257.000000,100.000000,69.296465


In [293]:
arc_tract = CreateSingleLevelSmokeAlarmModel(arc.copy(),'Tract',ACS.copy())
arc_tract.describe()

,num_surveys,detectors_found_total,detectors_found_prc,detectors_found_CI,detectors_working_total,detectors_working_prc,detectors_working_CI
count,71949.000000,71949.000000,71949.000000,71949.000000,71949.000000,71949.000000,71949.000000
mean,7.477296,5.002543,40.180386,10.497530,2.987297,23.098971,11.529151
std,26.903627,20.441452,41.663204,17.805888,15.152268,32.180938,18.587430
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,33.330000,0.000000,0.000000,0.000000,0.000000
75%,6.000000,4.000000,81.820000,17.518661,2.000000,42.860000,20.259933
max,4298.000000,3394.000000,100.000000,69.296465,2765.000000,100.000000,69.296465


In [294]:
arc_block = CreateSingleLevelSmokeAlarmModel(arc.copy(),'Block',ACS.copy())
arc_block.describe()

,num_surveys,detectors_found_total,detectors_found_prc,detectors_found_CI,detectors_working_total,detectors_working_prc,detectors_working_CI
count,213715.000000,213715.000000,213715.000000,213715.000000,213715.000000,213715.000000,213715.000000
mean,2.517296,1.684149,27.494668,6.498605,1.005699,15.763248,7.138333
std,10.225868,7.730288,40.568488,16.170726,5.641882,30.721666,16.925903
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,1.000000,66.670000,0.000000,1.000000,14.290000,0.000000
max,1863.000000,1546.000000,100.000000,69.296465,1325.000000,100.000000,69.296465


In [297]:
arc_tract

,num_surveys,detectors_found_total,detectors_found_prc,detectors_found_CI,detectors_working_total,detectors_working_prc,detectors_working_CI
01001020100,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000
01001020200,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000
01001020300,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000
01001020400,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000
01001020500,1.0,0.0,0.00,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...
56043000301,1.0,1.0,100.00,0.000000,1.0,100.0,0.000000
56043000302,2.0,2.0,100.00,0.000000,1.0,50.0,69.296465
56045951100,0.0,0.0,0.00,0.000000,0.0,0.0,0.000000
56045951300,12.0,8.0,66.67,26.671555,3.0,25.0,24.500000


In [295]:
# save single_level_models
for df, geo in [(arc_state, 'State'),(arc_county, 'County'),(arc_tract, 'Tract'),(arc_block, 'Block')] :
    df = df.copy()
    name = 'SmokeAlarmModel' + geo + '.csv'
    df.index.name = 'geoid'
    df.index =  '#_' + df.index 
    out_path =  path / 'Data' /'Model Outputs'/'Smoke_Alarm_Single_Level' / name
    df.to_csv(out_path)


In [301]:
sum(arc_county['num_surveys'] < 30)  

1499

In [255]:
all_IDS = arc_block.index

block_data =  arc_block[arc_block['num_surveys'] >= 30]
block_data['geography'] = 'block'
block_data.index.name = 'geoid'


<ipython-input-255-449182a23ba8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  block_data['geography'] = 'block'


In [266]:
remaining_ids = all_IDS[~all_IDS.isin(block_data.index)]
remaining_ids = remaining_ids.to_frame()
remaining_ids = remaining_ids.rename({0:'geoid'},axis = 1)

MultiLevelModel = block_data


for  geo,geo_len,df in [('tract',11,arc_tract), ('county',5,arc_county), ('state',2,arc_state)]:

    # find all remaining ids that are not in the block data 
    
    geo_index = remaining_ids

    # set up data index 
    
    geo_index['temp_geoid'] = geo_index.index.str[:geo_len]
    geo_index = geo_index.set_index('geoid')
    
    # create data set at one level
    geo_data = geo_index.merge(df, how = 'left', right_index = True, left_on = 'temp_geoid')
    geo_data = geo_data[geo_data['num_surveys'] > 30] 
    geo_data = geo_data.drop('temp_geoid',axis = 1 )
    geo_data['geography'] = geo
    # add to multilevel index
    MultiLevelModel = MultiLevelModel.append(geo_data)
    
    # update remaining_ids
    
    remaining_ids = remaining_ids[~remaining_ids.index.isin(MultiLevelModel.index)]
    del geo_index, geo_data



In [267]:
MultiLevelModel = MultiLevelModel.reset_index()
MultiLevelModel['geoid'] = '#_' + MultiLevelModel['geoid']

In [268]:
out_path =  path / 'Data' /'Model Outputs'/ 'SmokeAlarmModelOutput.csv'

MultiLevelModel.to_csv(out_path)